In [1]:
# import pymysql
import pandas as pd

### The following script will generate a SQL engine to communicate with the local MySQL database. The queries that follow can benefit from the full power of Python programming.

In [2]:
%run keys.py

In [3]:
df_fac = pd.read_sql_table('Facilities',engine)
df_fac.head(1)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200


In [4]:
df_book = pd.read_sql_table('Bookings',engine)
df_book.head(1)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2


In [5]:
df_mem = pd.read_sql_table('Members',engine)
df_mem.head(1)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00


### Some of the facilities at the country club charge a fee to members, but some do not. The following query lists the names of the facilities that do charge a fee. 

In [6]:
query = '''
SELECT name AS Facility, guestcost AS Cost
FROM Facilities
WHERE guestcost > 0
'''
q1 = pd.read_sql_query(query, engine)
q1

,Facility,Cost
0,Tennis Court 1,25.0
1,Tennis Court 2,25.0
2,Badminton Court,15.5
3,Table Tennis,5.0
4,Massage Room 1,80.0
5,Massage Room 2,80.0
6,Squash Court,17.5
7,Snooker Table,5.0
8,Pool Table,5.0


### How many facilities do not charge a fee to members?

In [7]:
query = '''
SELECT COUNT(membercost) AS no_member_cost
FROM Facilities
WHERE membercost = 0
'''
q2 = pd.read_sql_query(query, engine)
q2

,no_member_cost
0,4


### The following query produces a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Returning the facility id (facid), facility name, member cost, and monthly maintenance of the facilities in question.

In [8]:
query = '''
SELECT facid AS facility_id, 
       name AS facility_name, 
       membercost AS member_cost,
       monthlymaintenance AS monthly_maintenance
FROM Facilities
WHERE membercost < 0.2 * monthlymaintenance AND
      membercost > 0
'''
q3 = pd.read_sql_query(query, engine)
q3

,facility_id,facility_name,member_cost,monthly_maintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


### The following query retrieves the details of facilities with ID 1 and 5?

In [9]:
query = '''
SELECT *
FROM Facilities
WHERE facid IN (1, 5)
'''
q4 = pd.read_sql_query(query, engine)
q4

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


### This query produces a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Returning the name and monthly maintenance of the facilities in question.

In [10]:
query = '''
SELECT name AS Facility, 
       monthlymaintenance AS 'Monthly Maintenance',
       CASE WHEN monthlymaintenance <= 100 THEN 'cheap'
            ELSE 'expensive'
            END AS 'Cost Description'
FROM Facilities
'''
q5 = pd.read_sql_query(query, engine)
q5

,Facility,Monthly Maintenance,Cost Description
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


### Queries the first and last name of the last member(s) who signed up.

In [11]:
query = '''
SELECT firstname,
       surname
 FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members)
'''
q6 = pd.read_sql_query(query, engine)
q6

,firstname,surname
0,Darren,Smith


### The following query produces a list of all members who have used a tennis court. Included in the output is; the name of the court, and the name of the member formatted as a single column. Ensuring no duplicate data, and ordered by the member name.

In [12]:
query = '''
SELECT f.name AS facility,
       CONCAT(m.firstname, ' ', m.surname) AS name
FROM Bookings b
    JOIN Facilities f
      ON f.facid = b.facid
    JOIN Members m 
      ON m.memid = b.memid
WHERE f.facid IN (0,1)
GROUP BY 1,2
ORDER BY 2
'''
q7 = pd.read_sql_query(query, engine)
q7

,facility,name
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 2,David Farrell
8,Tennis Court 1,David Farrell
9,Tennis Court 1,David Jones


### The following produces a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remembering that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Included in the output is; the name of the facility, the name of the member formatted as a single column, and the cost. Ordered by descending cost, and this particular query does not use subqueries.

In [13]:
query = '''
SELECT f.name AS facility,
       CONCAT(m.firstname, ' ', m.surname) AS name,
       CASE WHEN b.memid = '0' THEN (b.slots * f.guestcost)
            ELSE (b.slots * f.membercost) END AS cost
FROM Bookings b
    JOIN Facilities f
      ON f.facid = b.facid
    JOIN Members m 
      ON m.memid = b.memid
WHERE b.starttime LIKE '2012-09-14%%'
  AND ((m.memid =0 AND b.slots * f.guestcost > 30)
	OR (m.memid !=0 AND b.slots * f.membercost > 30))
ORDER BY 3 DESC
'''
q8 = pd.read_sql_query(query, engine)
q8

,facility,name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


### This query, produces the same result as the previous query, but using a subquery.

In [14]:
query = '''
SELECT member, facility, cost
FROM (

SELECT CONCAT(m.surname, ', ', m.firstname) AS member, 
       f.name AS facility, 
       CASE WHEN m.memid =0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
            END AS cost
      FROM `Members` m
      JOIN `Bookings` b 
        ON m.memid = b.memid
INNER JOIN `Facilities` f 
        ON b.facid = f.facid
WHERE b.starttime LIKE '2012-09-14%%') AS bookings
	
WHERE cost > 30
ORDER BY cost DESC
'''
q9 = pd.read_sql_query(query, engine)
q9

,member,facility,cost
0,"GUEST, GUEST",Massage Room 2,320.0
1,"GUEST, GUEST",Massage Room 1,160.0
2,"GUEST, GUEST",Massage Room 1,160.0
3,"GUEST, GUEST",Massage Room 1,160.0
4,"GUEST, GUEST",Tennis Court 2,150.0
5,"GUEST, GUEST",Tennis Court 1,75.0
6,"GUEST, GUEST",Tennis Court 1,75.0
7,"GUEST, GUEST",Tennis Court 2,75.0
8,"GUEST, GUEST",Squash Court,70.0
9,"Farrell, Jemima",Massage Room 1,39.6


### The following query produces a list of facilities with a total revenue less than $1000. The output includes the facility name and total revenue, sorted by revenue. Remembering that there's a different cost for guests and members!

In [15]:
query = '''
SELECT f.name AS facility,
       SUM(CASE WHEN b.memid = '0' THEN (b.slots * f.guestcost)
            ELSE (b.slots * f.membercost) END) AS revenue
FROM Bookings b
    JOIN Facilities f
      ON f.facid = b.facid
GROUP BY 1
HAVING revenue < 1000
ORDER BY 2 DESC
'''
q10 = pd.read_sql_query(query, engine)
q10

,facility,revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0
